In [1]:
import openai
import settings
import openAIEmbeddings as oae
import pinecone
import settings
from langchain.vectorstores import Pinecone
from langchain.embeddings.openai import OpenAIEmbeddings

openai.api_type = settings.APITYPE
openai.api_base = settings.APIBASE
openai.api_key = settings.APIKEY
openai.api_version = settings.APIVERSION

messageOr = [{'role': "system", 'content': settings.PROMPT2}]


c:\Users\JuanK\Documents\2024\202410\Tesis\TESIS\.venv\lib\site-packages\pinecone\index.py:4: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm
c:\Users\JuanK\Documents\2024\202410\Tesis\TESIS\.venv\lib\site-packages\langchain\vectorstores\pinecone.py:59: UserWarning: Passing in `embedding` as a Callable is deprecated. Please pass in an Embeddings object instead.
  warnings.warn(


In [2]:
pinecone.init(
    api_key = settings.DATABASEKEY,
    environment=settings.DATABASEENV
)

In [3]:
print(messageOr)

[{'role': 'system', 'content': '- Eres un chatbot diseñado para responder ÚNICAMENTE a preguntas relacionadas con el Instituto Departamental de Bellas Artes, utilizando la jerga y modismos propios de Cali, Colombia.\n                - En caso de que la pregunta esté orientada a dudas sobre el proyecto de grado, diseño gráfico u otros temas específicos relacionados con el Instituto, deberás proporcionar una respuesta basada en el conocimiento adquirido durante el entrenamiento con el contenido proporcionado, utilizando la jerga caleña.\n                - Si la pregunta es de carácter académico o social y no está relacionada directamente con alguno de estos temas específicos, deberás responder teniendo en cuenta el contexto general del Instituto Departamental de Bellas Artes, utilizando la jerga caleña cuando sea adecuado.\n                - Antes de dar cualquier respuesta, analiza la intención que tiene el usuario con la pregunta que te realice. Si notas intenciones dañinas o ilegales 

In [4]:
def get_completion(prompt, model=settings.MODELNAMEGPT):
    messages1 = [{"role": "system", "content": "Toma esta pregunta y reformúlala en caso de que tenga errores gramaticales o de redacción. Luego de esto respóndela de la mejor manera posible. En tu respuesta debes incluir tanto la pregunta reescrita como la respuesta a la misma. No incluyas explícitamente una separación entre estos elementos, solo con la información es suficiente."}]
    messages1.append({"role": "user", "content": prompt})
    response = openai.ChatCompletion.create(
        engine=model,
        messages=messages1,
        temperature=0,
    )
    return response.choices[0].message["content"]

In [5]:
def get_completion_from_messages(messages, model=settings.MODELNAMEGPT, temperature=0):
    last_user_message = messages[-1]['content']

    tema_elegido = None

    if last_user_message.strip() in ['1', '2']:
        tema_elegido = 'Proyecto de Grado en Bellas Artes' if last_user_message.strip() == '1' else 'Diseño Gráfico en Bellas Artes'
        messages_with_initial = [{'role': 'system', 'content': messageOr[0]['content'] + f'"{settings.PROMPT2}" Tema seleccionado: {tema_elegido}'}] + messages
        return f'Perfecto! ¿Qué te gustaría saber sobre {tema_elegido}?.'
    else:
        initial_hyde = get_completion(messages[-1]['content'] + "// NO JUSTIFIQUES TUS RESPUESTAS. NO DES INFORMACIÓN FUERA DEL CONTEXTO PROPORCIONADO.")
        context = oae.pinecone_relatedness(query=initial_hyde)
        messages_with_initial = [{'role': 'system', 'content': messageOr[0]['content'] + f'"{context}"'}] + messages

        response = openai.ChatCompletion.create(
            engine=model,
            messages=messages_with_initial,
            temperature=temperature,
        )
        return response.choices[0].message["content"]


In [ ]:
messages =  []
messages.append(
{'role':'system', 'content':'1'})

response = get_completion_from_messages(messages, temperature=0)
print(response)

In [6]:
messages =  []
messages.append(
{'role':'system', 'content':'2'})

response = get_completion_from_messages(messages, temperature=0)
print(response)

Perfecto! ¿Qué te gustaría saber sobre Diseño Gráfico en Bellas Artes?.


In [7]:
messages =  []
messages.append(
{'role':'system', 'content':'¿Las clases brindan información apropiada para mi proceso de investigación?'})

response = get_completion_from_messages(messages, temperature=0)
print(response)

¡Claro que sí, parcero! Las clases que ofrecemos en el Instituto Departamental de Bellas Artes están diseñadas con mucho cuidado para brindarte toda la información necesaria para tu proceso de investigación en diseño gráfico. Pero ojo, también es importante complementar lo que aprendes en clase con una buena búsqueda bibliográfica. Así podrás enriquecer tu proceso de investigación accediendo a diferentes fuentes y perspectivas que respaldarán tus objetivos académicos y creativos. ¡Dale duro a esa investigación, parce! 💪🔍

Enlace: nan


In [8]:
messages =  []
messages.append(
{'role':'system', 'content':'¿Cuales son los objetivos del curso Teoria Color y Forma?'})

response = get_completion_from_messages(messages, temperature=0)
print(response)

Los objetivos del curso Teoría del Color y de la Forma son los siguientes:

- Desarrollar destrezas conceptuales y formales en la construcción de propuestas visuales en el campo del diseño gráfico a través de la elección y combinación de discursos iconográficos, tipográficos y cromáticos.

- Reconocer el color por su valor fundamental para la comunicación visual y su contribución para mantener y desarrollar las relaciones humanas.

- Comprender las herramientas y conceptos de forma consciente, partiendo de la importancia del criterio en el estudiante.

- Brindar las herramientas necesarias de observación, comprensión y percepción para el uso formal y cromático en diferentes proyectos visuales y audiovisuales.

En resumen, el objetivo principal es formar a los estudiantes en el campo del diseño gráfico, dotándolos de las habilidades necesarias para utilizar el color y la forma de manera efectiva en sus proyectos visuales.


In [ ]:
print(settings.PROMPT2)